### Library

In [116]:
from datetime import datetime, timedelta
import oandapy
import sys
sys.path.append('/Users/toshio/project/fx')
from config import token, acc_id
from lib.inference import Inference
import time

In [111]:
def get_current_state():
    try:
        response = oanda.get_position(acc_id, instrument ='USD_JPY')
        side = response.get('side')
        units = response.get('units')
    except oandapy.OandaError:
        side = 'zero_position'
        units = 0
    return side, units

In [112]:
def reset_position(side, units):
    trade_expire = datetime.utcnow() + timedelta(hours=1)
    trade_expire = trade_expire.isoformat("T") + "Z"
    
    response = oanda.create_order(acc_id,
        instrument='USD_JPY',
        units=units,
        side=side,
        type='limit',
        price=price,
        expiry=trade_expire
    )

In [113]:
def transaction(side, units):
    trade_expire = datetime.utcnow() + timedelta(minutes=10)
    trade_expire = trade_expire.isoformat("T") + "Z"
    
    response = oanda.create_order(acc_id,
        instrument='USD_JPY',
        units=units,
        side=side,
        type='limit',
        price=price,
        expiry=trade_expire
    )    

In [115]:
def episode():
    #現在のポジションを取得
    current_side, current_units = get_current_state()

    #ゼロポジションにする
    if current_side == 'buy':
        print('now you are long position')
        reset_position('sell', current_units)
        print('finished reset position')
    elif current_side == 'sell':
        print('now you are short position')
        reset_position('buy', current_units)
        print('finished reset position')
    elif current_side == 'zero_position':
        print('now you are zero position')

    #1時間後の差額を推定する
    inf = Inference()
    pred_delta=float(inf.pred[0][0])
    print('1時間の推定差額は{}'.format(pred_delta))


    if pred_delta >= 0.1:
        print('Buy!')
        transaction('buy', 100)
    elif pred_delta <= -0.1:
        print('Sell!')
        transaction('sell', 100)
    elif -0.1 < pred_delta < 0.1:
        print('do nothing')

In [122]:
#100時間取引する
j = 0
while j < 100:
    episode()
    time.sleep(60)
    j +=1

now you are zero position
1時間の推定差額は0.013195428065955639
do nothing
now you are zero position


KeyboardInterrupt: 